In [ ]:
import pandas as pd
import datetime

from bs4 import BeautifulSoup
import urllib

from IPython.display import Image, display
from IPython.core.display import HTML 

import warnings
warnings.filterwarnings('ignore')

### Scraping IGN game review information

In [ ]:
# After I checked how many pages for each game console, I create the below dict:
platformDict = {'xbox-one': 275,'ps4': 425,'nintendo-switch':50,'wii-u':125}
furl = 'http://www.ign.com/reviews/games/'
lurl = '?sortBy=reviewDate&sortOrder=desc&genre=&startIndex='

# Data that I would like to get:
gameCover = []
gameTitle = []
gamePlatform = []
gameGenre = []
gameRating = []
gamePhrase = []
gameReview = []

for x,y in platformDict.iteritems():
    url = furl+x+lurl
    pages = y
    for i in range(0,pages,25):
        surl = url + str(i)
        print surl
        r = urllib.urlopen(surl).read()
        soup = BeautifulSoup(r, "lxml")
        tr = soup.find_all("div",  class_ = "gameList-game")
        for x in tr:
            img = x.find('img') 
            # Only get review with game cover
            if img <> None:
                gameCover.append(img.get('src'))
                gameTitle.append(x.find('div', class_ = "game-title").find('a').text.strip())
                gamePlatform.append(x.find('div', class_ = "game-title").find('span').text.strip())
                gameGenre.append(x.find('span', class_ = "game-genre").string.replace(" ", "").strip())
                gameRating.append(float(x.find('div', class_ = "scoreBox").find('a').text.split('\n')[1]))
                gamePhrase.append(x.find('div', class_ = "scoreBox").find('a').text.split('\n')[2])
                gameReview.append(x.find('div', class_ = "scoreBox").find('a').get('href'))

                
columns = {'gameCover': gameCover,
           'gameTitle': gameTitle,
           'gamePlatform': gamePlatform,
           'gameGenre': gameGenre,
           'gameRating':gameRating,
           'gamePhrase':gamePhrase,
           'gameReview':gameReview
          }
df_IGN = pd.DataFrame(columns)

### Scraping IGN full game review

In [ ]:
%%time
# After got game review link
# then scrap the full game review
author = []
reviews = []
reviewLink = []

for x in df_IGN.gameReview.unique()[:]:
    surl = x
    r = urllib.urlopen(surl).read()
    soup = BeautifulSoup(r, "lxml")
    review = ''
    print surl
    reviewLink.append(x)
    # Some old review doesn't show author name
    if soup.find('span',class_ = "author-name") <> None:
        author.append(soup.find('span',class_ = "author-name").text.strip())
    else: 
        author.append('')    
    tr = soup.find_all("p")
    if tr <> None:
        for y in tr[:-2]:            
            review += ' ' + y.text.strip()
        reviews.append(review)
    else:
        reviews.append('')
        
columns = {'author': author,
           'reviews': reviews,
           'reviewLink': reviewLink
         }
df_reviews = pd.DataFrame(columns)
df_reviews.head()        

### Download Game Cover

In [ ]:
df = df_IGN.sort_values('gamePlatformAlias', ascending = True).groupby('gameReview').first()
poster_path = "gameCover/"
for x,y in df.sort_values('gameReviewID').iterrows():
    print y.gameReviewID,y.gameCover
    urllib.urlretrieve(y.gameCover, poster_path + str(y.gameReviewID) + ".jpg")

### Get IGN review user's top comment

Scraping user's comment isn't easy  
Its all loaded by Javascript   
cant just simply use urllib.urlopen(surl).read()  
In this case, wil try 'selenium' to actullay open a browser and scrap the comments

In [ ]:
# Use selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
IGN_link = df_IGN.gameReview.drop_duplicates().sort_values(ascending = False).tolist()
ign_comment = []
link = []

In [ ]:
browser = webdriver.Chrome('chromedriver.exe')

In [ ]:
## IGN use different class name on differnt game review
# check_loading = 'fyre-stream-sort-top-comments'
# check_loading = 'fyre-stream-sort-options'
check_loading = 'fyre-stream-more-container'

while len(IGN_link) > 0:
    url = IGN_link[0]
    print url
    browser.get(url)
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    print 'loading.......comment........'
    element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, check_loading)))
    element.click()
    comments = browser.find_elements_by_class_name('fyre-comment')
    print len(comments)
    gameComment = []
    if len(comments) > 0:
        for x in comments:
            gameComment.append(x.text)
    ign_comment.append(gameComment)
    link.append(url)
    IGN_link.pop(0)

In [ ]:
columns = {'ign_comment': ign_comment, 'link': link}
df_IGN_Comment = pd.DataFrame(columns)